# 5장 1절 수정 주가조회 패키지 비교

In [1]:
!pip install finance-datareader

In [2]:
!pip install pykrx

In [3]:
!pip install pandas_datareader

In [4]:
!pip install yfinance

In [5]:
import FinanceDataReader as fdr
from pykrx import stock
import pandas_datareader.data as pdr
import yfinance as yf

### 1. 상장종목 및 상장 폐지 종목 리스트 가져오기
#### 1.1 FinanceDataReader
- 현재 시점의 시장별 상장종목 리스트를 가져올 수 있음
- 종목코드, 시장, 종목명, 섹터, 산업군, 상장일, 결산월, 대표자, 홈페이지, 사업체 지역을 알 수 있음
- 과거 특정 시점의 상장종목 리스트는 알 수 없으나 상장폐지 종목을 조회할 수 있음
- https://github.com/FinanceData/FinanceDataReader

In [6]:
# 가장 최근 영업일의 시장별 종목리스트를 가져온다.
stocks = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 전체
stocks

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
8055,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8056,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
8057,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시
8058,419890,KOSPI,히어로즈 단기채권ESG액티브,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [7]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8060 entries, 0 to 8059
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Symbol          8060 non-null   object        
 1   Market          8060 non-null   object        
 2   Name            8060 non-null   object        
 3   Sector          2500 non-null   object        
 4   Industry        2483 non-null   object        
 5   ListingDate     2500 non-null   datetime64[ns]
 6   SettleMonth     2500 non-null   object        
 7   Representative  2500 non-null   object        
 8   HomePage        2346 non-null   object        
 9   Region          2500 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 692.7+ KB


#### 1.2 pykrx

In [8]:
# 날짜를 명시해주지 않으면 가장 최근 영업일의 시장별 종목리스트를 가져옴
tickers = stock.get_market_ticker_list()
tickers[:6], tickers[-6:] # 앞에서 6개, 뒤에서 6개 출력

(['095570', '006840', '027410', '282330', '138930', '001460'],
 ['079980', '005010', '000540', '000547', '000545', '003280'])

In [9]:
# 날짜를 명시해 주면 해당 날짜의 종목들이 반환된다.

tickers = stock.get_market_ticker_list('2010-01-01')
tickers[:6], tickers[-6:]

(['004560', '004565', '001460', '001465', '084680', '001040'],
 ['005010', '069260', '000540', '000547', '000545', '003280'])

In [10]:
stock.get_market_ticker_name('005930') # 종목명 조회   

'삼성전자'

### 2. 상장종목의 주가 조회
#### 2.1 FinanceDataReader
- 액면분할을 반영한 수정주가가 조회되는데 거래량은 보정되지 않은 상태임
- 조회 시점에 따라 조회할 수 있는 최대 과거 시점이 변동됨
- 예를 들어, 2022-03-22에 조회했을 때는 1998-02-05부터 조회됐었는데, 2022-03-23에 조회했을 때는 1998-02-06부터 조회됨 - 네이버금융에서 데이터를 가져오는데 일자가 지날 때 마다 과거 데이터를 삭제하는 것으로 보임

In [11]:
start_date = '1975-06-11'
end_date = '2022-05-03'

df_fdr = fdr.DataReader('005930', start=start_date, end=end_date)
df_fdr

,Open,High,Low,Close,Volume,Change
Date,,,,,,
1998-03-12,1606,1666,1588,1609,306558,NaN
1998-03-13,1608,1608,1515,1579,247920,-0.018645
1998-03-14,1525,1579,1525,1543,82450,-0.022799
1998-03-16,1543,1577,1541,1543,143731,0.000000
1998-03-17,1525,1534,1470,1485,233223,-0.037589
...,...,...,...,...,...,...
2022-04-27,65400,65500,64900,65000,18122084,-0.016641
2022-04-28,65400,65500,64500,64800,16895527,-0.003077
2022-04-29,65100,67600,65000,67400,26190390,0.040123


#### 2.2 pykrx
- 1990년 데이터부터 조회 가능
- 수정주가가 디폴트로 조회됨(adjusted 옵션으로 수정주가 여부 설정가능)
- adjusted=False로 조회하면 1995-05-02부터 조회되고, 거래대금과 등락률 칼럼이 추가됨
- 액면분할을 반영한 수정주가가 조회되는데 거래량은 보정되지 않은 상태임

In [12]:
start_date = '19750611'
end_date = '20220503'
df_pykrx = stock.get_market_ohlcv_by_date(fromdate=start_date, 
                                          todate=end_date,
                                          ticker="005930")
df_pykrx

,시가,고가,저가,종가,거래량
날짜,,,,,
1990-01-03,44000,45000,43200,44800,26240
1990-01-04,45000,45800,44800,45300,41900
1990-01-05,45000,45300,44300,44300,27400
1990-01-06,44800,45000,44500,44500,26380
1990-01-08,44500,44900,44000,44000,14790
...,...,...,...,...,...
2022-04-27,65400,65500,64900,65000,18122084
2022-04-28,65400,65500,64500,64800,16895527
2022-04-29,65100,67600,65000,67400,26190390


#### 2.3. pandas_datareader
- 데이터소스를 naver로 지정하면 FinanceDataReader와 유사한 기간으로 조회됨
- yahoo로 지정하면 2000년 데이터부터 조회할 수 있음
- 액면분할이 고려된 수정주가로 조회됨
- 네이버 조회시 수정되지 않은 거래량
- yahoo finance를 데이터소스로 사용할 때는 종목코드 뒤에 코스피 종목인 경우 .KS, 코스닥 종목인 경우 .KQ를 붙여줘야 함

In [13]:
df_pdr = pdr.DataReader('005930', 'naver', start=start_date, end = end_date)
df_pdr

,Open,High,Low,Close,Volume
Date,,,,,
1990-01-03,44000,45000,43200,44800,26240
1990-01-04,45000,45800,44800,45300,41900
1990-01-05,45000,45300,44300,44300,27400
1990-01-06,44800,45000,44500,44500,26380
1990-01-08,44500,44900,44000,44000,14790
...,...,...,...,...,...
2022-04-27,65400,65500,64900,65000,18122084
2022-04-28,65400,65500,64500,64800,16895527
2022-04-29,65100,67600,65000,67400,26190390


In [14]:
# yahoo finance를 데이터 소스로 사용할 때는 종목코드 뒤에 코스피 종목인 경우 .KS, 코스닥 종목인 경우 .KQ를 붙여줘야 함
df_pdr = pdr.DataReader('005930.KS', 'yahoo', start=start_date, end=end_date)
df_pdr

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-04,6110.0,5660.0,6000.0,6110.0,74195000.0,4651.738770
2000-01-05,6060.0,5520.0,5800.0,5580.0,74680000.0,4248.231445
2000-01-06,5780.0,5580.0,5750.0,5620.0,54390000.0,4278.686035
2000-01-07,5670.0,5360.0,5560.0,5540.0,40305000.0,4217.779297
2000-01-10,5770.0,5580.0,5600.0,5770.0,46880000.0,4392.886230
...,...,...,...,...,...,...
2022-04-27,65500.0,64900.0,65400.0,65000.0,18122084.0,65000.000000
2022-04-28,65500.0,64500.0,65400.0,64800.0,16895527.0,64800.000000
2022-04-29,67600.0,65000.0,65100.0,67400.0,26190390.0,67400.000000


#### 2.4. yfinance
- yahoo finance를 데이터 소스로 사용하는 패키지이기 때문에 위에서 설명한 바와 같이 코스피 종목에는 .KS, 코스닥 종목에는 .KQ를 붙여줘야 함
- 가장 다양한 정보 확인 가능 - 배당정보, 분할정보, 재무정보 등
- 2000년 데이터부터 조회 가능
- end에 설정한 일자의 전일자까지 조회되기 때문에 조회하고자 하는 종료일+1일을 end에 넣어줘야함
- 분단위 데이터도 조회 가능하지만 조회 가능한 기간에 제한이 있음

In [15]:
start_date = '1975-06-11'
end_date = '2022-05-02'

ticker = yf.Ticker('005930.KS')

ticker.history(
               interval='1d',
               start=start_date,
               end=end_date,
               actions=True,
               auto_adjust=True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-04,4567.990807,4651.737305,4309.137994,4651.737305,74195000,0.0,0.0
2000-01-05,4415.724949,4613.671240,4202.552020,4248.231934,74680000,0.0,0.0
2000-01-06,4377.658201,4400.498156,4248.231784,4278.685059,54390000,0.0,0.0
2000-01-07,4233.005937,4316.752457,4080.739536,4217.779297,40305000,0.0,0.0
2000-01-10,4263.457404,4392.883789,4248.230770,4392.883789,46880000,0.0,0.0
...,...,...,...,...,...,...,...
2022-04-25,66500.000000,66700.000000,66300.000000,66300.000000,11016474,0.0,0.0
2022-04-26,66400.000000,66700.000000,66100.000000,66100.000000,12946923,0.0,0.0
2022-04-27,65400.000000,65500.000000,64900.000000,65000.000000,18122084,0.0,0.0


- yfinance의 과거 주가는 다른 3개와 다름.
- 액면분할에 따른 거래량은 yfinance만 맞는듯.
- 우리는 가격에 주목하여 FinanceDataReader를 사용하기로 한다.

### 4. 상장폐지종목
- 상장폐지 종목의 주가 조회는 FinanceDataReader, pykrx, pandas_datareader에서 데이터 소스를 naver로 설정했을 때만 가능하다.
- yahoo finance에서는 상장폐지 종목에 대한 정보를 제공하지 않고 있는 것 같다.
- 상장폐지 종목의 리스트는 FinanceDataReader를 통해 조회하면 된다.

#### 4.1. FinanceDataReader

In [16]:
# KRX stock delisting symbol list 상장폐지 종목 전체 리스트
krx_delisting = fdr.StockListing('KRX-DELISTING')
krx_delisting

,Symbol,Name,Market,SecuGroup,Kind,ListingDate,DelistingDate,Reason,ArrantEnforceDate,ArrantEndDate,Industry,ParValue,ListingShares,ToSymbol,ToName
0,06031012,3S R,KOSDAQ,신주인수권증서,보통주,2012-05-14,2012-05-21,,NaT,NaT,,0.0,1194422.0,,
1,00684014,AK홀딩스8R,KOSPI,신주인수권증서,보통주,2014-07-28,2014-08-04,,NaT,NaT,,0.0,1278299.0,,
2,13893015,BNK금융지주 8R,KOSPI,신주인수권증서,보통주,2015-12-24,2016-01-05,,NaT,NaT,,0.0,55969410.0,,
3,13893014,BS금융지주5R,KOSPI,신주인수권증서,보통주,2014-06-18,2014-06-25,,NaT,NaT,,0.0,32791220.0,,
4,03204017,C&S자산관리 34R,KOSDAQ,신주인수권증서,보통주,2017-02-02,2017-02-09,,NaT,NaT,,0.0,3995063.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3373,034370,럭키소재,KOSPI,주권,,1979-06-23,1991-11-11,해산 사유 발생,NaT,NaT,,NaN,NaN,,
3374,028460,태평양건설,KOSPI,주권,,1977-06-09,1991-10-05,영업활동정지 6월 계속,1991-08-28,1991-10-04,,NaN,NaN,,
3375,028450,금성투자금융,KOSPI,주권,,1986-08-29,1991-09-02,해산 사유 발생,NaT,NaT,,NaN,NaN,,
3376,028440,삼화,KOSPI,주권,,1974-06-12,1991-07-12,감사의견 의견거절,1991-06-05,1991-07-11,,NaN,NaN,,


#### 4.2 Pykrx

In [17]:
start_date = '19980312'
end_date = '20220502'
stock.get_market_ohlcv_by_date(fromdate=start_date, 
                               todate=end_date,
                               ticker="001047")

,시가,고가,저가,종가,거래량
날짜,,,,,
1999-01-29,6059,6816,5346,6682,47560
1999-02-01,6237,6727,6014,6593,27720
1999-02-02,6594,6683,6594,6637,28480
1999-02-03,6594,6683,6059,6058,26780
1999-02-04,6059,6505,6059,6504,17050
...,...,...,...,...,...
2009-03-13,32650,32650,32650,32650,0
2009-03-16,32650,32650,32650,32650,0
2009-03-17,32650,32650,32650,32650,0


#### 4.3 Pandas_DataReader

In [18]:
df_pdr = pdr.DataReader('001047', 'naver', start=start_date, end=end_date)
df_pdr

,Open,High,Low,Close,Volume
Date,,,,,
1999-01-29,6059,6816,5346,6682,47560
1999-02-01,6237,6727,6014,6593,27720
1999-02-02,6594,6683,6594,6637,28480
1999-02-03,6594,6683,6059,6058,26780
1999-02-04,6059,6505,6059,6504,17050
...,...,...,...,...,...
2009-03-13,32650,32650,32650,32650,0
2009-03-16,32650,32650,32650,32650,0
2009-03-17,32650,32650,32650,32650,0


# 5장 2절 수정 주가 데이터 수집(FinanceDataReader 이용)

In [19]:
import FinanceDataReader as fdr

In [20]:
stock_list = fdr.StockListing('KRX').dropna() # 상장폐지 종목은 삭제

In [21]:
stock_list = stock_list.head(3) # 시간 단축을 위하여 3개만 활용

In [22]:
stock_list

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시


In [23]:
import time
import pandas as pd

In [24]:
daily = pd.DataFrame()
for code, name in stock_list[['Symbol', 'Name']].values:
    print(code, name)
    date = '2022-05-02'
    ohlcv = fdr.DataReader(code, date, date) # 하루치 데이터 불러오기
    ohlcv['Code'] = code
    ohlcv['Name'] = name
    daily = pd.concat([daily, ohlcv])

060310 3S
095570 AJ네트웍스
006840 AK홀딩스


[Pandas 기초] 데이터프레임 합치기(merge, join, concat)
- https://yganalyst.github.io/data_handling/Pd_12/

Pandas(판다스) - FinanceDataReader을 사용해 주식 데이터를 dataframe으로 불러오기 
- https://blog.naver.com/PostView.naver?blogId=ji0eeeee&logNo=221527166066&parentCategoryNo=&categoryNo=11&viewDate=&isShowPopularPosts=true&from=search

열 추가, 값 수정, 데이터 합치기
- https://nalara12200.tistory.com/162 

[python] 팬더스 데이터 프레임의 인덱스를 열로 변환하는 방법은 무엇입니까?
- http://daplus.net/python-%ED%8C%AC%EB%8D%94-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%94%84%EB%A0%88%EC%9E%84%EC%9D%98-%EC%9D%B8%EB%8D%B1%EC%8A%A

In [25]:
daily

,Open,High,Low,Close,Volume,Change,Code,Name
Date,,,,,,,,
2022-05-02,3200,3275,3200,3250,69707,0.003086,060310,3S
2022-05-02,6630,6660,6420,6650,30618,0.000000,095570,AJ네트웍스
2022-05-02,21550,21650,21050,21350,14068,-0.006977,006840,AK홀딩스


In [26]:
import pandas as pd
import numpy as np
import time, json, requests
from datetime import datetime, timedelta, date

import pymysql # python에서 mysql을 사용하는 패키지

import sqlalchemy # sql 접근 및 관리를 도와주는 패키지
from sqlalchemy import create_engine

In [27]:
server = '127.0.0.1'
user = 'root'
passwd = 'abcd123456'
db = 'whsf1_db'

# sqlalchemy의 create_engine을 이용하여 DB 연결
engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user, passwd, server, db))


In [32]:
daily.to_sql(name='kospi_adjusted1',con=engine,if_exists='append',index=False,
             dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있다.

                 'Open' : sqlalchemy.types.BIGINT(),
                 'High' : sqlalchemy.types.BIGINT(),
                 'Low' : sqlalchemy.types.BIGINT(),
                 'Close' : sqlalchemy.types.BIGINT(),
                 'Volume' : sqlalchemy.types.BIGINT(),
                 'Change' : sqlalchemy.types.FLOAT(),
                 'Code' : sqlalchemy.types.VARCHAR(10),
                 'Name' : sqlalchemy.types.TEXT(),
                 'Date' : sqlalchemy.types.DATE(),
             }
            )

In [33]:
# DB 데이터 저장 확인
sql = 'select * from kospi_adjusted1'
db_data = pd.read_sql(sql, engine)
db_data

,Open,High,Low,Close,Volume,Change,Code,Name
0,3200,3275,3200,3250,69707,0.003086,060310,3S
1,6630,6660,6420,6650,30618,0.000000,095570,AJ네트웍스
2,21550,21650,21050,21350,14068,-0.006977,006840,AK홀딩스


In [34]:
# DB 접속 해제
engine.dispose()
#cur.close()
#conn.close()